# <center> Предсказание победителя в Dota 2
<center> <img src="https://meduza.io/impro/YnJZAHUW6WHz_JQm1uRPkTql_qAhbfxt3oFJLGH7CJg/fill/980/0/ce/1/aHR0cHM6Ly9tZWR1/emEuaW8vaW1hZ2Uv/YXR0YWNobWVudHMv/aW1hZ2VzLzAwNy8x/NTcvNjk1L29yaWdp/bmFsL0tMVThLbUti/ZG5pSzlibDA0Wmlw/WXcuanBn.webp" width="700" height="700">

[Почитать подбробнее](https://meduza.io/feature/2021/10/19/rossiyskaya-komanda-vyigrala-chempionat-mira-po-dota-2-i-poluchila-18-millionov-dollarov-postoyte-otkuda-takie-dengi-neuzheli-igrat-v-dotu-tak-slozhno)

#### [Оригинальная статья](https://arxiv.org/pdf/2106.01782.pdf)
    
### Начало

Посмотрим на готовые признаки и сделаем первую посылку. 

1. [Описание данных](#Описание-данных)
2. [Описание признаков](#Описание-признаков)
3. [Наша первая модель](#Наша-первая-модель)
4. [Посылка](#Посылка)

### Первые шаги на пути в датасайенс

5. [Кросс-валидация](#Кросс-валидация)
6. [Что есть в json файлах?](#Что-есть-в-json-файлах?)
7. [Feature engineering](#Feature-engineering)

### Импорты

In [ ]:
import os
import json
import pandas as pd
import datetime
import warnings
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

import xgboost
import catboost
import lightgbm as lgb
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline

In [ ]:
SEED = 10801
sns.set_style(style="whitegrid")
plt.rcParams["figure.figsize"] = 12, 8
warnings.filterwarnings("ignore")

## <left>Описание данных

Файлы:

- `sample_submission.csv`: пример файла-посылки
- `train_raw_data.jsonl`, `test_raw_data.jsonl`: "сырые" данные 
- `train_data.csv`, `test_data.csv`: признаки, созданные авторами
- `train_targets.csv`: результаты тренировочных игр

## <left>Описание признаков
    
Набор простых признаков, описывающих игроков и команды в целом

In [ ]:
df_train_features = pd.read_csv("../input/new-dataset/train_data.csv", index_col="match_id_hash")
df_train_targets = pd.read_csv("../input/new-dataset/train_targets.csv", index_col="match_id_hash")

<span style="color:purple"></span>

<span style="color:purple">Добавлю в ноутбук описание признаков в таблице</span>

Статистику игроков (префикс 'r' для команды Radiant, 'd' для Dire), которая включает поля:

hero_id - id героя

K/D/A: количество убийств, смертей и помощи.

lh - last hits - число убитых вражеских крипов.

denies - число добитых собственных крипов с целью не дать врагу золото.

gold - количество золота у персонажа.

xp - количество опыта у персонажа.

level - уровень героя.

health and max_health - здоровье и максимальное значение здоровья.

max_mana - максимальное количество маны (мана нужна для использования способностей).

x, y - текущие координаты игрока на карте.

stuns - сколько секунд герой продержал врагов в оглушении.

camps_stacked - сложная механика для первого раза, но чем она выше, тем проще команде получать золото.

rune_pickups - число найденных рун (они дают разные временные бонусы, например, увеличение урона).

firstblood_claimed - сделал ли игрок первое убийство в игре (обычно за это дают больше золота).

teamfight_participation - в каком проценте сражений поучаствовал персонаж.

towers_killed - число разрушенных вышек.

roshans_killed - число убитых Рошанов (дает возможность одному из героев команды сразу возродиться после сметри).

obs_placed, sen_placed - число Observer и Sentry вардов, установленных игроком (первые дают обзор на карте, а вторые показывают невидимых героев при наличии обзора в данной области, а также вражеские варды).

В JSON файлах содержится гораздо больше информации, которую вам предстоит найти и с помощью нее создать дополнительные признаки. 

<span style="color:purple">В таблице отсуствуют пропуски значений</span>

In [ ]:
df_train_features.shape

In [ ]:
df_train_features.isna().sum().sum()

In [ ]:
df_train_features.head()

<span style="color:purple">Посмотрим на признаки, которые есть в нашей таблице. Получается, что описанные выше параметры повторяются для каждого игрока в каждом матче (r1-r5, d1-d5)</span>

In [ ]:
for col in df_train_features.columns:
    print(col)

Имеем ~32 тысячи наблюдений, каждое из которых характеризуется уникальным `match_id_hash` (захэшированное id матча), и 245 признаков. `game_time` показывает момент времени, в который получены эти данные. То есть по сути это не длительность самого матча, а например, его середина, таким образом, в итоге мы сможем получить модель, которая будет предсказывать вероятность победы каждой из команд в течение матча (хорошо подходит для букмекеров).

Нас интересует поле `radiant_win` (так называется одна из команд, вторая - dire). Остальные колоки здесь по сути получены из "будущего" и есть только для тренировочных данных, поэтому на них можно просто посмотреть).

In [ ]:
df_train_targets.head()

<span style="color:purple">Можем сказать, что наш тренировочный датасет сбалансирован, поскольку у нас примерно равное количество матчей, в которых radiant выиграл или проиграл, и значит, у нас нет дисбаланса классов</span>

In [ ]:
df_train_targets["radiant_win"].value_counts()

## <left>Наша первая модель

In [ ]:
X = df_train_features.values
y = df_train_targets["radiant_win"].values.astype("int8")

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.3, 
                                                      random_state=SEED)

#### Обучим случайный лес

In [ ]:
%%time
rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
rf_model.fit(X_train, y_train)

#### Сделаем предсказания и оценим качество на отложенной части данных

In [ ]:
y_pred = rf_model.predict_proba(X_valid)[:, 1]

In [ ]:
valid_score = roc_auc_score(y_valid, y_pred)
print("ROC-AUC score на отложенной части:", valid_score)

Посмотрим на accuracy:

In [ ]:
valid_accuracy = accuracy_score(y_valid, y_pred > 0.5)
print("Accuracy score (p > 0.5) на отложенной части:", valid_accuracy)

## <left>Кросс-валидация

Во многих случаях кросс-валидация оказывается лучше простого разбиения на test и train. Воспользуемся `ShuffleSplit` чтобы создать 5 70%/30% наборов данных.

In [ ]:
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=SEED)

In [ ]:
%%time
rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
cv_scores_rf = cross_val_score(rf_model, X, y, cv=cv, scoring="roc_auc")

In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")

## <span style="color:purple">Catboost</span>

<span style="color:purple">Воспользуемся *catboost*, отлично показавшим себя на одной из последних лекций:</span>

In [ ]:
cat = catboost.CatBoostRegressor(iterations=200, learning_rate=0.1, depth=6, random_state=SEED, min_data_in_leaf=7, verbose=0)

In [ ]:
%%time
cat.fit(X_train, y_train)

In [ ]:
y_pred = cat.predict(X_valid)

In [ ]:
cat_valid_score = roc_auc_score(y_valid, y_pred)
print("ROC-AUC score на отложенной части:", cat_valid_score)

In [ ]:
cat_valid_accuracy = accuracy_score(y_valid, y_pred > 0.5)
print("Accuracy score (p > 0.5) на отложенной части:", cat_valid_accuracy)

<span style="color:purple">При использовании catboost значения ROC-AUC и accuracy score были чуть лучше, чем при использовании описанного выше базового леса. Воспользуемся кросс-валидацией:</span>

In [ ]:
%%time
cat = catboost.CatBoostRegressor(iterations=200, learning_rate=0.1, depth=6, random_state=SEED, min_data_in_leaf=7, verbose=0)
cv_scores_rf = cross_val_score(cat, X, y, cv=cv, scoring="roc_auc")

In [ ]:
print(f"Среднее значение ROC-AUC на кросс-валидации: {cv_scores_rf.mean()}")

<span style="color:purple">При использовании catboost и кросс-валидации среднее значение ROC-AUC опять же было чуть лучше, чем у нашего базового леса и catboost, обученного на разбиении тренировочных данных. Попробуем подобрать параметры для нашей модели с помощью `GridSearchCV`</span>

In [ ]:
cat = catboost.CatBoostRegressor(random_state=42, verbose=0, learning_rate=0.1)
param = {"iterations" : range(50, 250, 50), "depth" : range(5, 11), "min_data_in_leaf" : range(5, 11)}
grid_search = GridSearchCV(cat, param, cv=cv, scoring="roc_auc")

In [ ]:
grid_search.fit(X_train, y_train)

<span style="color:purple">Когда запустила grid_search с iterations = 3000 и learning_rate = 0.05:</span>

<img src ="https://miro.medium.com/max/1400/1*owTKhjo8cf85piCC4sqLoQ.jpeg" width="640" length="426">

In [ ]:
grid_search.best_params_

In [ ]:
best_cat = grid_search.best_estimator_
y_pred = best_cat.predict(X_valid)

In [ ]:
cat_valid_score = roc_auc_score(y_valid, y_pred)
print("ROC-AUC score на отложенной части:", cat_valid_score)

<span style="color:purple">Я не стала перезапускать еще раз (считалось примерно 3,5 часа). И странно, что у меня величина ROC-AUC с лучшими параметрами модели получилась меньше той, что была без подбора параметров</span>.

## <span style="color:purple">Stacking</span>

<span style="color:purple">Попробуем разобраться со стекингом и рассмотрим несколько моделей. Воспользуемся [примером стекинга](https://github.com/Dyakonov/ml_hacks/blob/master/dj_stacking.ipynb), который давали на лекции:</span>

<span style="color:purple">Авторская реализация стекинга:</span>

In [ ]:
class DjStacking(BaseEstimator, ClassifierMixin):  
    """Стэкинг моделей scikit-learn"""

    def __init__(self, models, ens_model):
        """
        Инициализация
        models - базовые модели для стекинга
        ens_model - мета-модель
        """
        self.models = models
        self.ens_model = ens_model
        self.n = len(models)
        self.valid = None
        
    def fit(self, X, y=None, p=0.3, cv=5, err=0.001, random_state=None):
        """
        Обучение стекинга
        p - в каком отношении делить на обучение / тест
            если p = 0 - используем всё обучение!
        cv  (при p=0) - сколько фолдов использовать
        err (при p=0) - величина случайной добавки к метапризнакам
        random_state - инициализация генератора
            
        """
        if (p > 0): # делим на обучение и тест
            # разбиение на обучение моделей и метамодели
            train, valid, y_train, y_valid = train_test_split(X, y, test_size=p, random_state=random_state)
            
            # заполнение матрицы для обучения метамодели
            self.valid = np.zeros((valid.shape[0], self.n))
            for t, clf in enumerate(self.models):
                clf.fit(train, y_train)
                self.valid[:, t] = clf.predict(valid)
                
            # обучение метамодели
            self.ens_model.fit(self.valid, y_valid)
            
        else: # используем всё обучение
            
            # для регуляризации - берём случайные добавки
            self.valid = err*np.random.randn(X.shape[0], self.n)
            
            for t, clf in enumerate(self.models):
                # это oob-ответы алгоритмов
                self.valid[:, t] += cross_val_predict(clf, X, y, cv=cv, n_jobs=-1, method='predict')
                # но сам алгоритм надо настроить
                clf.fit(X, y)
            
            # обучение метамодели
            self.ens_model.fit(self.valid, y)  
            

        return self
    


    def predict(self, X, y=None):
        """
        Работа стэкинга
        """
        # заполение матрицы для мета-классификатора
        X_meta = np.zeros((X.shape[0], self.n))
        
        for t, clf in enumerate(self.models):
            X_meta[:, t] = clf.predict(X)
        
        a = self.ens_model.predict(X_meta)
        
        return (a)

<span style="color:purple">Создание и обучение классификаторов:</span>

In [ ]:
def run(clf, X, y, label):
    a = clf.predict(X)  
    print (label + ' AUC-ROC  = ' + str( roc_auc_score(y, a) ))

<span style="color:purple">Базовые классификаторы (также была идея подобрать оптимальные параметры для каждой базовой модели, но мне не хватило времени, и я не знаю, улучшается ли качество конечной модели при улучшении ее "компонентов" подбором параметров. Я предполагаю, что должно. Но это увеличивает время работы самого алгоритма: я просто вручную поставила параметры побольше и у меня kaggle задумался на 2 часа и так и не выдал результат):</span>

In [ ]:
knn1 = KNeighborsRegressor(n_neighbors=3)
knn1.fit(X_train, y_train)
run(knn1, X_valid, y_valid, '3NN')

knn2 = KNeighborsRegressor(n_neighbors=10)
knn2.fit(X_train, y_train)
run(knn2, X_valid, y_valid, '10NN')

rg0 = Ridge(alpha=0.01)
rg0.fit(X_train, y_train)
run(rg0, X_valid, y_valid, 'ridge-0.01')

rg1 = Ridge(alpha=1.1)
rg1.fit(X_train, y_train)
run(rg1, X_valid, y_valid, 'ridge-1.1')

rg2 = Ridge(alpha=100.1)
rg2.fit(X_train, y_train)
run(rg2, X_valid, y_valid, 'ridge-100.1')

rf1 = RandomForestRegressor(n_estimators=100, max_depth=2)
rf1.fit(X_train, y_train)
run(rf1, X_valid, y_valid, 'rf-d2')

rf2 = RandomForestRegressor(n_estimators=100, max_depth=5)
rf2.fit(X_train, y_train)
run(rf2, X_valid, y_valid, 'rf-d5')

gbm1 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=2, n_estimators=200, n_jobs=-1, objective='regression')    
gbm1.fit(X_train, y_train)
run(gbm1, X_valid, y_valid, 'gbm-d2')

gbm2 = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.05, max_depth=5, n_estimators=200, n_jobs=-1, objective='regression')    
gbm2.fit(X_train, y_train)
run(gbm2, X_valid, y_valid, 'gbm-d5')

xgb1 = xgboost.XGBRegressor(n_estimators = 200, max_depth = 2, learning_rate = 0.05, random_state=SEED)
xgb1.fit(X_train, y_train)
run(xgb1, X_valid, y_valid, 'xgb-d2')

xgb2 = xgboost.XGBRegressor(n_estimators = 200, max_depth = 5,  learning_rate = 0.05, random_state=SEED)
xgb2.fit(X_train, y_train)
run(xgb2, X_valid, y_valid, 'xgb-d5')

catb1 = catboost.CatBoostRegressor(iterations=200, learning_rate=0.05, depth=2, random_state=SEED, min_data_in_leaf=7, verbose=0)
catb1.fit(X_train, y_train)
run(catb1, X_valid, y_valid, 'cb-d2')

catb2 = catboost.CatBoostRegressor(iterations=200, learning_rate=0.05, depth=5, random_state=SEED, min_data_in_leaf=7, verbose=0)
catb2.fit(X_train, y_train)
run(catb2, X_valid, y_valid, 'cb-d5')

<span style="color:purple">Запускаем блендинг и стекинг:</span>

In [ ]:
models = [knn1, knn2, rg1, rg2, rf1, rf2, gbm1, gbm2, xgb1, xgb2, catb1, catb2]
ens_model = Ridge()
s1 = DjStacking(models, ens_model)
s1.fit(X_train, y_train)
run(s1, X_valid, y_valid, '1-stacking')
s2 = DjStacking(models, ens_model)
s2.fit(X_train, y_train, p=-1)
run(s2, X_valid, y_valid, '2-stacking')

<span style="color:purple">Получаем результат лучше, чем у базовых моделей</span>

<span style="color:purple">Несколько блендингов подряд:</span>

In [ ]:
ens_model = Ridge(0.001)
s1 = DjStacking(models, ens_model)
a = 0
for t in range(10):
    s1.fit(X_train, y_train, p=0.3)
    a += s1.predict(X_valid, y_train)
    
    auc = roc_auc_score(y_valid, a)
    print(auc)

<span style="color:purple">В примере предлагали варьировать число фолдов, но у меня считалось долго и качество предсказания по ROC-AUC не улучшилось:</span>

In [ ]:
ens_model = Ridge(0.001)

s1 = DjStacking(models, ens_model)
a = 0
for t in range(2, 11):
    s1.fit(X_train, y_train, p=-1, cv=t, err=0.001)
    a = s1.predict(X_valid, y_train)
    auc = roc_auc_score(y_valid, a)
    print(auc)

## <left>Что есть в json файлах?

Описание сырых данных можно найти в `train_matches.jsonl` и `test_matches.jsonl`. Каждый файл содержит одну запись для каждого матча в [JSON](https://en.wikipedia.org/wiki/JSON) формате. Его легко превратить в питоновский объект при помощи метода `json.loads`.

In [ ]:
with open("../input/new-dataset/train_raw_data.jsonl") as fin:
    # прочтем 419 строку
    for i in range(419):
        line = fin.readline()
    
    # переведем JSON в питоновский словарь 
    match = json.loads(line)

<span style="color:purple">Когда решила посмотреть, а что там в match:</span>

<img src ="https://cs8.pikabu.ru/post_img/big/2016/10/30/9/147783766317230953.jpg" width="700" length="350">

In [ ]:
match

<span style="color:purple">Мы можем узнать различную информацию для 9 игрока в этом матче, в частности, число убийств, смертей и помощи:</span>

In [ ]:
player = match["players"][9]
player["kills"], player["deaths"], player["assists"]

KDA - может быть неплохим признаком, этот показатель считается как:
    
<center>$KDA = \frac{K + A}{D}$

Информация о количестве использованных способностей:

In [ ]:
player["ability_uses"]

In [ ]:
for i, player in enumerate(match["players"]):
    plt.plot(player["times"], player["xp_t"], label=str(i+1))

plt.legend()
plt.xlabel("Time, s")
plt.ylabel("XP")
plt.title("XP change for all players");

#### Сделаем чтение файла с сырыми данными и добавление новых признаков удобным

В этот раз для чтение `json` файлов лучше использовать библиотеку `ujson`, иначе все будет слишком долго :(

<span style="color:purple">Установим ujson:</span>

In [ ]:
!pip install ujson

In [ ]:
try:
    import ujson as json
except ModuleNotFoundError:
    import json
    print ("Подумайте об установке ujson, чтобы работать с JSON объектами быстрее")
    
try:
    from tqdm.notebook import tqdm
except ModuleNotFoundError:
    tqdm_notebook = lambda x: x
    print ("Подумайте об установке tqdm, чтобы следить за прогрессом")

    
def read_matches(matches_file, total_matches=31698, n_matches_to_read=None):
    """
    Аргуент
    -------
    matches_file: JSON файл с сырыми данными
    
    Результат
    ---------
    Возвращает записи о каждом матче
    """
    
    if n_matches_to_read is None:
        n_matches_to_read = total_matches
        
    c = 0
    with open(matches_file) as fin:
        for line in tqdm(fin, total=total_matches):
            if c >= n_matches_to_read:
                break
            else:
                c += 1
                yield json.loads(line)

#### Чтение данных в цикле

Чтение всех данных занимает 1-2 минуты, поэтому для начала можно попробовать следующее:

1. Читать 10-50 игр
2. Написать код для работы с этими JSON объектами
3. Убедиться, что все работает
4. Запустить код на всем датасете
5. Сохранить результат в `pickle` файл, чтобы в следующий раз не переделывать все заново

<span style="color:purple">Сохраним вывод функции в pickle файл. Сделаем это для 1 матча, чтобы разобраться со структурой:</span>

In [ ]:
with open('data_1_match.pickle', 'wb') as f:
    for i in read_matches("../input/new-dataset/train_raw_data.jsonl", n_matches_to_read=1):
        pickle.dump(i, f)

<span style="color:purple">Посмотрим на содержимое:</span>

In [ ]:
obj = pd.read_pickle("data_1_match.pickle")
obj

<span style="color:purple">Сохраним вывод функции в pickle файл (для всех матчей):</span>

In [ ]:
with open('data.pickle', 'wb') as f:
    for i in read_matches("../input/new-dataset/train_raw_data.jsonl"):
        pickle.dump(i, f)

<span style="color:purple">И вроде бы, я всё сделала, как надо, но дальше не поняла, как дальше использовать pickle файл. Думала, что его нужно подавать его в input для функции, которая добавляет новые признаки (хотя в комментариях написано, что функция ждет json)</span>

## <left>Feature engineering

Напишем функцию, которая поможет нам легче добавлять новые признаки.

<span style="color:purple">Возможно, я не разобралась, но я не обращалась к слоту игрока, чтобы узнать, из какой он команды. Я просто проверила, что в каждом матче я могу обратиться к 10 игрокам, и если их вызывать по порядку, то и слоты их будут 0-1 и 128-132. И так повторяется в каждом матче. Т.е. первый по порядку всегда из radiant. Забавно будет (на самом деле, нет), если у меня тут ошибка.</span>

<span style="color:purple">И я также пробовала добавлять признаки не по командам, а по каждому игроку: иногда результат не отличался, иногда был хуже. Поэтому решила оставить для каждой из команд.</span>

In [ ]:
def add_new_features(df_features, matches_file):
    """
    Аргументы
    -------
    df_features: таблица с данными
    matches_file: JSON файл с сырыми данными
    
    Результат
    ---------
    Добавляет новые признаки в таблицу
    """
    
    for match in read_matches(matches_file):
        match_id_hash = match['match_id_hash']

        # Посчитаем количество разрушенных вышек обеими командами:
        radiant_tower_kills = 0
        dire_tower_kills = 0
        for objective in match["objectives"]:
            if objective["type"] == "CHAT_MESSAGE_TOWER_KILL":
                if objective["team"] == 2:
                    radiant_tower_kills += 1
                if objective["team"] == 3:
                    dire_tower_kills += 1

        df_features.loc[match_id_hash, "radiant_tower_kills"] = radiant_tower_kills
        df_features.loc[match_id_hash, "dire_tower_kills"] = dire_tower_kills
        df_features.loc[match_id_hash, "diff_tower_kills"] = radiant_tower_kills - dire_tower_kills
        
        # Посчитаем показатель KDA для каждой из команд. Рассчитаем как (K+A)/(D+1):        
        KDA_list = []
        KDA_r = 0
        KDA_d = 0
        for player in match["players"]:
            KDA_list.append((player["kills"] + player["assists"])/(player["deaths"] + 1))        
        
        df_features.loc[match_id_hash, "r_KDA"] = sum(KDA_list[0:5])
        df_features.loc[match_id_hash, "d_KDA"] = sum(KDA_list[5:10])
   
        # Посчитаем число использованных способностей обеими командами:
        r_ability_uses = 0        
        d_ability_uses = 0
        ability = []
        for player in match["players"]:
            player_abilities = 0
            for el in player["ability_uses"].values():
                player_abilities += el
            ability.append(player_abilities)

        df_features.loc[match_id_hash, "r_ability_uses"] = sum(ability[0:5])
        df_features.loc[match_id_hash, "d_ability_uses"] = sum(ability[5:10])
        
        # Посчитаем число использованных способностей обеими командами:
        r_item_uses = 0        
        d_item_uses = 0
        item_uses = []
        for player in match["players"]:
            player_item_uses = 0
            for el in player["item_uses"].values():
                player_item_uses += el
            item_uses.append(player_item_uses)

        df_features.loc[match_id_hash, "r_item_uses"] = sum(item_uses[0:5])
        df_features.loc[match_id_hash, "d_item_uses"] = sum(item_uses[5:10])
            
        # Посчитаем количество единиц урона, нанесенного каждой из команд
        r_damage = 0        
        d_damage = 0
        damage = []
        for player in match["players"]:
            player_damage = 0
            for el in player["damage"].values():
                player_damage += el
            damage.append(player_damage)

        df_features.loc[match_id_hash, "r_damage"] = sum(damage[0:5])
        df_features.loc[match_id_hash, "d_damage"] = sum(damage[5:10])
        
        # Посчитаем "исцеления"  
        r_healing = 0        
        d_healing = 0
        healing = []
        for player in match["players"]:
            player_healing = 0
            for el in player["healing"].values():
                player_healing += el
            healing.append(player_healing)

        df_features.loc[match_id_hash, "r_healing"] = sum(healing[0:5])
        df_features.loc[match_id_hash, "d_healing"] = sum(healing[5:10])
        
        # Посчитаем покупки для каждой из команд:
        r_purchase = 0        
        d_purchase = 0
        purchase = []
        for player in match["players"]:
            player_purchase = 0
            for el in player["purchase"].values():
                player_purchase += el
            purchase.append(player_purchase)

        df_features.loc[match_id_hash, "r_purchase"] = sum(purchase[0:5])
        df_features.loc[match_id_hash, "d_purchase"] = sum(purchase[5:10])
    

        # Посчитаем количество улучшенных способностей для каждой из команд:
        r_abilities_upg = 0        
        d_abilities_upg = 0
        ability_upgrades = []
        for player in match["players"]:
            ability_upgrades.append(len(player["ability_upgrades"]))

        df_features.loc[match_id_hash, "r_ability_upgrades"] = sum(ability_upgrades[0:5])
        df_features.loc[match_id_hash, "d_ability_upgrades"] = sum(ability_upgrades[5:10])
        
        # Посчитаем количество улучшенных способностей для каждой из команд:
        r_nearby_creep_death_count = 0        
        d_nearby_creep_death_count = 0
        nearby_creep_death_count = []
        for player in match["players"]:
            nearby_creep_death_count.append(player["nearby_creep_death_count"])

        df_features.loc[match_id_hash, "r_nearby_creep_death_count"] = sum(nearby_creep_death_count[0:5])
        df_features.loc[match_id_hash, "d_nearby_creep_death_count"] = sum(nearby_creep_death_count[5:10])
        
        # Посчитаем количество рун каждой из команд:        
        player["runes"]
        r_runes = 0        
        d_runes= 0
        runes = []
        for player in match["players"]:
            player_runes = 0
            for el in player["runes"].values():
                player_runes += el
            runes.append(player_runes)

        df_features.loc[match_id_hash, "r_runes"] = sum(runes[0:5])
        df_features.loc[match_id_hash, "d_runes"] = sum(runes[5:10])
        
        
        # ... (/¯◡ ‿ ◡)/¯☆*:・ﾟ добавляем новые признаки ...

In [ ]:
# Скопируем таблицу с признаками
df_train_features_extended = df_train_features.copy()

# Добавим новые
add_new_features(df_train_features_extended, "../input/new-dataset/train_raw_data.jsonl")

In [ ]:
df_train_features_extended.head()

In [ ]:
%%time
rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
cv_scores_base = cross_val_score(rf_model, X, y, cv=cv, scoring="roc_auc", n_jobs=-1)
print(f"ROC-AUC на кросс-валидации для базовых признаков: {cv_scores_base.mean()}")

In [ ]:
%%time
rf_model = RandomForestClassifier(n_estimators=300, max_depth=7, n_jobs=-1, random_state=SEED)
cv_scores_extended_base_ft = cross_val_score(rf_model, df_train_features_extended.values, y, cv=cv, scoring="roc_auc", n_jobs=-1)
print(f"ROC-AUC на кросс-валидации для новых признаков: {cv_scores_extended_base_ft.mean()}")

In [ ]:
%%time
cat = catboost.CatBoostRegressor(iterations=200, learning_rate=0.1, depth=6, random_state=SEED, min_data_in_leaf=7, verbose=0)
cv_scores_extended_cat = cross_val_score(cat, df_train_features_extended.values, y, cv=cv, scoring="roc_auc", n_jobs=-1)
print(f"ROC-AUC на кросс-валидации для новых признаков (catboost): {cv_scores_extended_cat.mean()}")

In [ ]:
X_sl = df_train_features_extended.values
y_sl = df_train_targets["radiant_win"].values.astype("int8")
X_sl_train, X_sl_valid, y_sl_train, y_sl_valid = train_test_split(X_sl, y_sl, test_size=0.3, random_state=SEED)

In [ ]:
ens_model = Ridge(0.001)
s3 = DjStacking(models, ens_model)
a = 0
for t in range(10):
    s3.fit(X_sl_train, y_sl_train, p=0.3)
    a += s3.predict(X_sl_valid)
    
    auc = roc_auc_score(y_sl_valid, a)
    print(auc)

In [ ]:
s3_valid_score = roc_auc_score(y_valid, a)
print("ROC-AUC score на отложенной части (блендинг):", s3_valid_score)

In [ ]:
ens_model = Ridge()
s4 = DjStacking(models, ens_model)
s4.fit(X_sl_train, y_sl_train, p=-1)
a = s4.predict(X_sl_valid)
s4_valid_score = roc_auc_score(y_sl_valid, a)

In [ ]:
print("ROC-AUC score на отложенной части (стекинг):", s4_valid_score)

In [ ]:
print(f"ROC-AUC на кросс-валидации для базовых признаков: {cv_scores_base.mean()}")
print(f"ROC-AUC на кросс-валидации для новых признаков: {cv_scores_extended_base_ft.mean()}")
print(f"ROC-AUC на кросс-валидации для новых признаков (catboost): {cv_scores_extended_cat.mean()}")
print(f"ROC-AUC на кросс-валидации для новых признаков (блендинг): {s3_valid_score}")
print(f"ROC-AUC на кросс-валидации для новых признаков (стекинг): {s4_valid_score}")

<span style="color:purple">Я ожидала, что раз уж при добавлении новых признаков в таблицу ROC-AUC для базовой модели возрастает (с 0,772 до 0,788), то этого же можно ожидать и для catboost, и для стекинга. Однако результаты на новых данных даже хуже, чем на старых. Пришлось сделать сабмит предсказаний по исходной таблице</span>

## <left>Посылка

In [ ]:
df_test_features = pd.read_csv("../input/new-dataset/test_data.csv", 
                                   index_col="match_id_hash")

X_test = df_test_features.values
y_test_pred = s1.predict(X_test)

df_submission = pd.DataFrame({"radiant_win_prob": y_test_pred}, 
                                 index=df_test_features.index)

In [ ]:
submission_filename = "submission_{}.csv".format(
    datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
df_submission.to_csv(submission_filename)
print("Файл посылки сохранен, как: {}".format(submission_filename))

## <span style="color:purple">Therapy time</span>

<span style="color:purple">Спасибо, было интересно, хоть я и расстроилась немного, что добавленные в таблице новые данные улучшили мне ROC-AUC базового леса, но все равно ранее посчитанный ROC-AUC для нескольких блендингов оказался лучше. Кажется, что прошлась по верхам, хотя можно было бы и искать другие параметры, копнув чуть глубже. В любом случае, это первый опят соревнований на Kaggle, вроде даже за score baseline перевалила :)</span>

<img src ="https://funart.pro/uploads/posts/2021-07/1627199139_45-funart-pro-p-ustavshii-kotik-zhivotnie-krasivo-foto-63.jpg" width="504" length="504">